# Análisis Cuantitativo — Contexto y Objetivos
Este notebook calcula y visualiza métricas de calidad del aire a partir de series horarias por estación y contaminante.

Objetivos:
- Preparar los datos en formato largo (fecha-hora, estación, contaminante, valor).
- Calcular métricas mensuales por estación/contaminante (AUC, media, p50, p90, máximo diario, horas válidas).
- Estimar IAQI/AQI diarios siguiendo reglas de la US EPA (PM2.5, O3, CO, NO2).
- Agregar a nivel ciudad y generar visualizaciones (tendencias y distribución por categorías).

Convenciones y supuestos clave:
- Frecuencia base: 1 hora (se re-muestrea a 1H cuando se requiere).
- Cobertura mínima para medias móviles/diarias: 75% de datos válidos en la ventana (p. ej., ≥6 de 8 horas).
- Unidades: O3 se convierte de ppb a ppm para IAQI (ppm = ppb/1000).
- Archivo de entrada principal: `data/processed/panel_BALANCED_MAIN_JanJul_2020_2024_2025_AB_v1.csv`.


## Métricas Mensuales por Estación/Contaminante
Se calculan a partir de la serie horaria en 1H por estación y contaminante dentro de cada mes:
- AUC: área bajo la curva por regla del trapecio con paso 1H; suma de 0.5·(v_t + v_{t+1}).
- mean: media mensual aproximada como `AUC / valid_hours`.
- p50, p90: percentiles 50 y 90 de los valores horarios del mes.
- max_diario: máximo de los máximos diarios dentro del mes.
- valid_hours: número de datos horarios no nulos en el mes.

Salida: `reports/metrics_monthly.csv` con columnas `[station, pollutant, month, auc, mean, p50, p90, max_diario, valid_hours]`.


## IAQI y AQI (US EPA)
Para cada estación se estiman IAQI diarios por contaminante y el AQI final del día como el máximo IAQI disponible:
- PM2.5: promedio diario 24h; se requiere ≥18 h válidas (75%). Se trunca/ajusta según tablas antes del IAQI.
- O3: máximo diario de medias móviles 8h; cada ventana requiere ≥6 datos válidos (75%). Para IAQI se usa O3 en ppm.
- CO: máximo diario de medias móviles 8h; ventana con ≥6 datos.
- NO2: máximo diario 1h; se requiere cobertura suficiente por día.
- AQI diario (por estación): `max(IAQI_PM2.5, IAQI_O3, IAQI_CO, IAQI_NO2)`.

Agregación a ciudad: se toma el máximo entre estaciones por fecha. Además, se calcula una media móvil de 7 días (`aqi_ma7`) y se categorizan los días según los umbrales EPA. La tabla resultante se guarda en `reports/aqi_daily.csv` y la versión agregada de ciudad se usa para los gráficos.


## Visualizaciones e Interpretación
- Métricas mensuales (2020, 2024, 2025 Ene–Jul): se grafican por contaminante la mediana entre estaciones y el IQR (Q25–Q75) para AUC y media. Archivos: `reports/figs/monthly_metrics_*.png`.
- AQI ciudad en el tiempo: serie diaria con `aqi` y media móvil 7d (`aqi_ma7`) para 2020/2024/2025 Ene–Jul. Archivos: `reports/figs/aqi_analysis/AQI_timeline_city_*.png`.
- Días por categoría AQI: barras apiladas por año (conteo y porcentaje) con clasificación EPA. Archivos: `reports/figs/aqi_analysis/AQI_days_by_category_city*.png`.

Notas de lectura:
- AUC resume carga mensual; la media aproxima el nivel típico horario.
- La mediana e IQR entre estaciones mitigan outliers y muestran dispersión espacial.
- El máximo entre estaciones para AQI ciudad refleja la peor condición diaria.

## Estructura de los datos de entrada
Columnas principales del panel:
- `date`: fecha-hora (UTC/local según fuente) a frecuencia 1H.
- `station_code`: código de estación.
- Contaminantes horarios: `PM2.5`, `NO2`, `CO`, `O3`, `NO`, `NOX_final`.
- Indicadores de imputación (booleanos): `*_AB_imputed` indican si el valor fue imputado.
- Campos auxiliares: `year`, `month`, `dow`, `period_window` según el panel.
Los pasos siguientes convierten a formato largo para facilitar el cálculo de métricas.


In [17]:
import os, pandas as pd, numpy as np
import matplotlib.pyplot as plt

DATA_DIR = os.path.join("..","data","processed")
IN_FILE  = os.path.join(DATA_DIR, "panel_BALANCED_MAIN_JanJul_2020_2024_2025_AB_v1.csv")

df = pd.read_csv(IN_FILE, parse_dates=["date"], engine="pyarrow")

df.head()



,date,station_code,period_window,year,month,dow,PM2.5,NO2,CO,O3,NO,NOX_final,PM2.5_AB_imputed,NO2_AB_imputed,CO_AB_imputed,O3_AB_imputed,NO_AB_imputed,NOX_final_AB_imputed
0,2024-01-01 00:00:00,CE,baseline_JanJul2024,2024,1,0,NaN,20.55,1.91,19.5,4.15,26.6,True,True,True,True,True,True
1,2024-01-01 01:00:00,CE,baseline_JanJul2024,2024,1,0,NaN,35.30,2.53,14.0,5.70,41.0,True,False,False,False,False,False
2,2024-01-01 02:00:00,CE,baseline_JanJul2024,2024,1,0,NaN,24.90,2.16,19.0,4.00,28.9,False,False,False,False,False,False
3,2024-01-01 03:00:00,CE,baseline_JanJul2024,2024,1,0,NaN,21.00,1.92,20.0,3.50,24.5,False,False,False,False,False,False
4,2024-01-01 04:00:00,CE,baseline_JanJul2024,2024,1,0,NaN,17.50,1.74,25.0,3.50,21.0,False,False,False,False,False,False


In [19]:
# Convertir df a formato largo: datetime, station, pollutant, value
pollutant_cols = ['PM2.5','NO2','CO','O3','NO','NOX_final']
present = [c for c in pollutant_cols if c in df.columns]

long_df = (
    df.rename(columns={'date': 'datetime', 'station_code': 'station'})
      .melt(id_vars=['datetime', 'station'],
            value_vars=present,
            var_name='pollutant',
            value_name='value')
)

# Ignorar filas con NaN en value
long_df = long_df.dropna(subset=['value'])

long_df.head()


,datetime,station,pollutant,value
5065,2024-01-01 00:00:00,NE,PM2.5,999.0
5066,2024-01-01 01:00:00,NE,PM2.5,464.0
5067,2024-01-01 02:00:00,NE,PM2.5,690.0
5068,2024-01-01 03:00:00,NE,PM2.5,729.0
5069,2024-01-01 04:00:00,NE,PM2.5,557.0


### Nota: Cálculo de métricas mensuales
- Entrada: `long_df` con columnas `datetime`, `station`, `pollutant`, `value`.
- Paso horario: se fuerza a `1H` y se ordena.
- AUC: suma de trapecios consecutivos con ambos valores no nulos.
- valid_hours: conteo de valores horarios válidos en el mes.
- mean: `AUC / valid_hours` cuando `valid_hours > 0`.
- p50/p90: percentiles 50 y 90 de los valores horarios del mes.
- max_diario: máximo del máximo diario en el mes.
- Salida: `reports/metrics_monthly.csv`.


In [20]:
# Métricas mensuales por estación y contaminante usando AUC (trapecio, paso 1h)
import numpy as np
import pandas as pd
import os

ldf = long_df.copy()
ldf['datetime'] = pd.to_datetime(ldf['datetime'])
ldf['month'] = ldf['datetime'].dt.to_period('M').dt.to_timestamp()

def _monthly_metrics(g):
    s = g.set_index('datetime').sort_index().asfreq('1H')
    v = s['value']
    valid_hours = int(v.notna().sum())
    m = v.notna() & v.shift(-1).notna()
    auc = float((0.5 * (v[m] + v.shift(-1)[m])).sum())
    mean_monthly = (auc / valid_hours) if valid_hours > 0 else np.nan
    arr = v.to_numpy()
    p50 = float(np.nanpercentile(arr, 50)) if valid_hours > 0 else np.nan
    p90 = float(np.nanpercentile(arr, 90)) if valid_hours > 0 else np.nan
    daily_max = s['value'].resample('1D').max(min_count=1)
    max_diario = float(daily_max.max()) if daily_max.notna().any() else np.nan
    return pd.Series({
        'auc': auc,
        'mean': mean_monthly,
        'p50': p50,
        'p90': p90,
        'max_diario': max_diario,
        'valid_hours': valid_hours,
    })

metrics_monthly = (
    ldf.groupby(['station','pollutant','month'], as_index=False)
       .apply(_monthly_metrics)
       .reset_index(drop=True)
)

# Guardar CSV
out_dir = os.path.join('..','reports','tables')
os.makedirs(out_dir, exist_ok=True)
out_file = os.path.join(out_dir, 'metrics_monthly.csv')
metrics_monthly.to_csv(out_file, index=False)
out_file, metrics_monthly.head()

C:\Users\facun\AppData\Local\Temp\ipykernel_19848\704974064.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  s = g.set_index('datetime').sort_index().asfreq('1H')
C:\Users\facun\AppData\Local\Temp\ipykernel_19848\704974064.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_monthly_metrics)


('..\\reports\\tables\\metrics_monthly.csv',
   station pollutant      month        auc      mean    p50    p90  max_diario  \
 0      CE        CO 2020-01-01  1916.3700  2.575766  2.545  3.777        5.34   
 1      CE        CO 2020-02-01  1789.2250  2.570726  2.540  3.035        4.55   
 2      CE        CO 2020-03-01  2421.8000  3.255108  3.260  3.730        4.53   
 3      CE        CO 2020-04-01   630.7875  0.876094  0.810  1.190        3.54   
 4      CE        CO 2020-05-01  1072.4275  1.441435  1.460  1.700        2.36   
 
    valid_hours  
 0        744.0  
 1        696.0  
 2        744.0  
 3        720.0  
 4        744.0  )

### Nota: IAQI/AQI diarios (EPA)
- PM2.5: promedio diario 24h; requiere ≥18 h válidas por día (75%).
- O3 y CO: medias móviles 8h; cada ventana requiere ≥6 datos válidos (75%).
- NO2: máximo diario de 1h; requiere cobertura suficiente por día.
- Conversión: O3 en ppm para IAQI (`ppm = ppb/1000`).
- IAQI: se calcula con `iaqi_from_bp` usando tablas de puntos de quiebre EPA.
- AQI diario (por estación): máximo IAQI disponible entre contaminantes.
- Salidas: `reports/aqi_daily.csv`; posteriormente se agregará a nivel ciudad.


In [21]:
# IAQI diario para PM2.5 (24h) y O3 (máx 8h móvil) con reglas US EPA
# Supuesto: O3 en ppb -> convertir a ppm dividiendo por 1000 antes de IAQI
import numpy as np
import pandas as pd
import os

ldf = long_df.copy()
ldf['datetime'] = pd.to_datetime(ldf['datetime'])
ldf = ldf.set_index('datetime').sort_index()

def truncate(x, decimals):
    factor = 10 ** decimals
    return np.floor(x * factor) / factor

# Breakpoints EPA
PM25_BREAKPOINTS = [
    (0.0, 12.0, 0, 50),
    (12.1, 35.4, 51, 100),
    (35.5, 55.4, 101, 150),
    (55.5, 150.4, 151, 200),
    (150.5, 250.4, 201, 300),
    (250.5, 350.4, 301, 400),
    (350.5, 500.4, 401, 500),
]

O3_8H_BREAKPOINTS = [
    (0.000, 0.054, 0, 50),
    (0.055, 0.070, 51, 100),
    (0.071, 0.085, 101, 150),
    (0.086, 0.105, 151, 200),
    (0.106, 0.200, 201, 300),
]

CO_8H_BREAKPOINTS = [
    (0.0, 4.4, 0, 50),
    (4.5, 9.4, 51, 100),
    (9.5, 12.4, 101, 150),
    (12.5, 15.4, 151, 200),
    (15.5, 30.4, 201, 300),
    (30.5, 40.4, 301, 400),
    (40.5, 50.4, 401, 500),
]

NO2_1H_BREAKPOINTS = [
    (0, 53, 0, 50),
    (54, 100, 51, 100),
    (101, 360, 101, 150),
    (361, 649, 151, 200),
    (650, 1249, 201, 300),
    (1250, 1649, 301, 400),
    (1650, 2049, 401, 500),
]

def iaqi_from_bp(c, bps):
    if pd.isna(c):
        return np.nan
    for Clow, Chigh, Ilow, Ihigh in bps:
        if c <= Chigh:
            return (Ihigh - Ilow) / (Chigh - Clow) * (c - Clow) + Ilow
    Clow, Chigh, Ilow, Ihigh = bps[-1]
    return (Ihigh - Ilow) / (Chigh - Clow) * (c - Clow) + Ilow

# ---- PM2.5: promedio diario 24h, requiere >= 18 horas válidas (75%) ----
pm = ldf[ldf['pollutant'] == 'PM2.5'].copy()
pm_results = []
for station, g in pm.groupby('station'):
    s = g['value'].asfreq('1H')
    daily_count = s.resample('1D').count()
    daily_mean = s.resample('1D').mean()
    daily_mean = daily_mean.where(daily_count >= 18)
    conc_24h = truncate(daily_mean, 1)
    iaqi_pm25 = conc_24h.apply(lambda x: iaqi_from_bp(x, PM25_BREAKPOINTS))
    iaqi_pm25 = iaqi_pm25.round(0)
    pm_results.append(pd.DataFrame({
        'station': station,
        'date': conc_24h.index,
        'iaqi_pm25': iaqi_pm25
    }))
pm25_daily = pd.concat(pm_results, ignore_index=True) if pm_results else pd.DataFrame(columns=['station','date','iaqi_pm25'])

# ---- O3: máximo diario de medias móviles 8h, cada 8h requiere >= 6 datos (75%) ----
o3 = ldf[ldf['pollutant'] == 'O3'].copy()
o3_results = []
for station, g in o3.groupby('station'):
    s = g['value'].asfreq('1H')
    s_ppm = s / 1000.0  # ppb -> ppm
    o3_8h = s_ppm.rolling(window=8, min_periods=6).mean()
    o3_8h = truncate(o3_8h, 3)
    daily_max8h = o3_8h.resample('1D').max()
    iaqi_o3 = daily_max8h.apply(lambda x: iaqi_from_bp(x, O3_8H_BREAKPOINTS))
    iaqi_o3 = iaqi_o3.round(0)
    o3_results.append(pd.DataFrame({
        'station': station,
        'date': daily_max8h.index,
        'iaqi_o3': iaqi_o3
    }))
o3_daily = pd.concat(o3_results, ignore_index=True) if o3_results else pd.DataFrame(columns=['station','date','iaqi_o3'])

# ---- CO: máximo diario de medias móviles 8h, cada 8h requiere >= 6 datos (75%)) ----
co = ldf[ldf['pollutant'] == 'CO'].copy()
co_results = []
for station, g in co.groupby('station'):
    s = g['value'].asfreq('1H')
    co_8h = s.rolling(window=8, min_periods=6).mean()
    co_8h = truncate(co_8h, 1)  # ppm truncado a 0.1
    daily_max8h = co_8h.resample('1D').max()
    iaqi_co = daily_max8h.apply(lambda x: iaqi_from_bp(x, CO_8H_BREAKPOINTS))
    iaqi_co = iaqi_co.round(0)
    co_results.append(pd.DataFrame({
        'station': station,
        'date': daily_max8h.index,
        'iaqi_co': iaqi_co
    }))
co_daily = pd.concat(co_results, ignore_index=True) if co_results else pd.DataFrame(columns=['station','date','iaqi_co'])

# ---- NO2: máximo diario 1h, requiere >= 18 horas válidas en el día ----
no2 = ldf[ldf['pollutant'] == 'NO2'].copy()
no2_results = []
for station, g in no2.groupby('station'):
    s = g['value'].asfreq('1H')  # ppb
    daily_count = s.resample('1D').count()
    max_1h = s.resample('1D').max()
    max_1h = max_1h.where(daily_count >= 18)
    conc_1h = truncate(max_1h, 0)  # ppb truncado a entero
    iaqi_no2 = conc_1h.apply(lambda x: iaqi_from_bp(x, NO2_1H_BREAKPOINTS))
    iaqi_no2 = iaqi_no2.round(0)
    no2_results.append(pd.DataFrame({
        'station': station,
        'date': conc_1h.index,
        'iaqi_no2': iaqi_no2
    }))
no2_daily = pd.concat(no2_results, ignore_index=True) if no2_results else pd.DataFrame(columns=['station','date','iaqi_no2'])

# ---- AQI final del día: máximo IAQI disponible (PM2.5, O3, CO, NO2) ----
aqi_daily = (pm25_daily
             .merge(o3_daily, on=['station','date'], how='outer')
             .merge(co_daily, on=['station','date'], how='outer')
             .merge(no2_daily, on=['station','date'], how='outer'))
aqi_daily['aqi'] = aqi_daily[['iaqi_pm25','iaqi_o3','iaqi_co','iaqi_no2']].max(axis=1, skipna=True)
aqi_daily = aqi_daily.sort_values(['station','date']).reset_index(drop=True)

# Guardar CSV
out_dir = os.path.join('..','reports','tables')
os.makedirs(out_dir, exist_ok=True)
out_file = os.path.join(out_dir, 'aqi_daily.csv')
aqi_daily.to_csv(out_file, index=False)
out_file, aqi_daily.head()

C:\Users\facun\AppData\Local\Temp\ipykernel_19848\3344883949.py:67: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  s = g['value'].asfreq('1H')
C:\Users\facun\AppData\Local\Temp\ipykernel_19848\3344883949.py:85: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  s = g['value'].asfreq('1H')
C:\Users\facun\AppData\Local\Temp\ipykernel_19848\3344883949.py:103: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  s = g['value'].asfreq('1H')
C:\Users\facun\AppData\Local\Temp\ipykernel_19848\3344883949.py:120: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  s = g['value'].asfreq('1H')  # ppb


('..\\reports\\tables\\aqi_daily.csv',
   station       date  iaqi_pm25  iaqi_o3  iaqi_co  iaqi_no2   aqi
 0      CE 2020-01-01        NaN     23.0     44.0       9.0  44.0
 1      CE 2020-01-02        NaN     24.0     41.0      15.0  41.0
 2      CE 2020-01-03        NaN     44.0     41.0      14.0  44.0
 3      CE 2020-01-04        NaN     40.0     35.0      16.0  40.0
 4      CE 2020-01-05        NaN     48.0     36.0      19.0  48.0)

### Nota: Gráficos de métricas mensuales
- Fuente: `reports/metrics_monthly.csv`.
- Agregación: para cada contaminante y mes, mediana e IQR (Q25–Q75) entre estaciones.
- Se generan dos gráficos por contaminante: AUC mensual y media mensual (ambos con mediana e IQR).
- Cobertura temporal: 2020, 2024 y 2025 (enero–julio).
- Uso: facilita comparar estacionalidad y cambios interanuales mitigando outliers.


In [22]:
out_dir = os.path.join('..','reports','figs','monthly_metrics')

In [23]:
out_dir = os.path.join('..','reports','figs','aqi_analysis')

### Nota: AQI timeline (qué se grafica)
- Contaminantes para IAQI: PM2.5 (promedio diario 24 h), O3 (máx de medias móviles de 8 h, en ppm para IAQI), CO (máx de medias móviles de 8 h), NO2 (máx 1 h).
- AQI diario por estación = máximo de los IAQI disponibles (PM2.5, O3, CO, NO2).
- AQI de ciudad (timeline) = máximo de AQI entre estaciones por fecha.
- Se grafica la serie diaria `aqi` y su media móvil de 7 días `aqi_ma7`.
- Menor valor = mejor calidad de aire; picos representan eventos con mayor riesgo.

### Nota: AQI de ciudad y categorías
- Agregación ciudad: máximo de AQI entre estaciones por fecha; se calcula media móvil 7d (`aqi_ma7`).
- Gráficos: líneas temporales por año (2020/2024/2025 Ene–Jul) y barras de días por categoría (conteo y porcentaje).
- Archivos: `reports/figs/aqi_analysis/AQI_timeline_city_*.png`, `reports/figs/aqi_analysis/AQI_days_by_category_city*.png`.
- Interpretación: las categorías sintetizan riesgo sanitario; el máximo entre estaciones refleja la peor condición diaria.

In [24]:
out_dir = os.path.join('..','reports','figs','aqi_analysis')

### Notas de cálculo y lectura: comparativas mensuales (deltas vs 2020 y mapas de calor)\n
- Línea base y emparejamiento mensual: para cada mes `m` se compara el valor de un año con el valor del mismo mes en 2020 (mes-a-mes, no un promedio del semestre).\n
- Definición de delta (vs 2020): `Δ(año, m) = valor(año, m) − valor(2020, m)`.\n
  - Para AQI: `valor` es la mediana mensual del AQI de ciudad; el AQI de ciudad por día es el máximo entre estaciones. Unidades: puntos AQI.\n
  - Para métricas mensuales `mean` por contaminante: `valor` es la mediana entre estaciones del promedio horario mensual. Unidades: las del contaminante en el panel (p. ej., PM2.5 en µg/m³; NO/NO2/NOX en ppb; CO en ppm; O3 en ppb). La conversión de O3 a ppm solo aplica al cálculo de IAQI, no a estas métricas.\n
- Signo e interpretación: `Δ < 0` indica mejora frente a 2020 (menor es mejor); `Δ > 0` indica empeoramiento; `Δ ≈ 0` sin cambio.\n
- Celdas sin dato: si falta el valor base de 2020 para un mes/contaminante, el delta queda `NaN` y la celda aparece en blanco.\n
- Cómo leer el mapa de calor: filas = años de comparación (2024, 2025); columnas = mes (1–12).\n
  - Paleta y centro: `RdYlGn_r` centrada en 0 → verde = negativo (mejora), rojo = positivo (empeora), amarillo ≈ 0.\n
  - Anotaciones: el número en cada celda es la magnitud del cambio vs 2020, en puntos AQI o en la unidad del contaminante según corresponda.\n
- Alcance temporal: las comparaciones se hacen con el rango de meses disponible en el panel (p. ej., Ene–Jul cuando aplica).\n
## Comparativa mensual 2020 vs 2024 vs 2025 (AQI y métricas)
En esta sección comparamos, mes a mes, si el aire mejoró o empeoró entre 2020, 2024 y 2025, usando: 
- AQI mensual de ciudad (agregación por máximo entre estaciones y mediana mensual).
- Métrica mensual `mean` (mediana entre estaciones por contaminante).

Notas: menor valor implica mejor calidad de aire (tanto AQI como concentraciones). Se reportan gráficos de línea por año y mapas de calor con el cambio vs 2020 (negativo = mejora).


In [38]:
%pip install seaborn

# AQI mensual de ciudad: comparativa 2020 vs 2024 vs 2025
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tables_dir = os.path.join('..','reports','tables')
figs_aqi_dir = os.path.join('..','reports','figs','aqi_analysis')
os.makedirs(figs_aqi_dir, exist_ok=True)

aqi_path = os.path.join(tables_dir, 'aqi_daily.csv')
aqi_daily = pd.read_csv(aqi_path, parse_dates=['date'])

# Agregación ciudad: máximo de AQI entre estaciones por fecha
city = (aqi_daily.groupby('date')['aqi'].max().rename('aqi_city').reset_index())
city['year'] = city['date'].dt.year
city['month_num'] = city['date'].dt.month

years = [2020, 2024, 2025]
city_f = city[city['year'].isin(years)].copy()

aqi_monthly = (city_f.groupby(['year','month_num'])['aqi_city']
                    .median()
                    .reset_index())
# Guardar tabla
out_csv = os.path.join(tables_dir, 'aqi_city_monthly_median.csv')
aqi_monthly.to_csv(out_csv, index=False)

pivot = aqi_monthly.pivot(index='month_num', columns='year', values='aqi_city').sort_index()

# Gráfico de líneas (comparativa por año)
plt.figure(figsize=(10,4))
pivot.plot(marker='o')
plt.title('AQI ciudad mediana mensual (máx estaciones) - 2020 vs 2024 vs 2025')
plt.xlabel('Mes')
plt.ylabel('AQI (menor es mejor)')
plt.xticks(range(1,13))
plt.grid(True, alpha=0.3)
plt.legend(title='Año')
plt.tight_layout()
plt.savefig(os.path.join(figs_aqi_dir, 'AQI_city_monthly_comparison.png'), dpi=150)
plt.close()

# Mapa de calor: cambio vs 2020 (negativo = mejora)
if 2020 in pivot.columns:
    delta_vs2020 = pivot.subtract(pivot[2020], axis=0)
    keep_cols = [c for c in [2024, 2025] if c in delta_vs2020.columns]
    if keep_cols:
        deltaT = delta_vs2020[keep_cols].T
        deltaT.index.name = 'year'
        delta_melt = deltaT.reset_index().melt(id_vars='year', var_name='month_num', value_name='delta_vs2020')
        delta_melt.to_csv(os.path.join(tables_dir, 'AQI_city_monthly_change_vs2020.csv'), index=False)
        plt.figure(figsize=(10,2.5))
        sns.heatmap(deltaT, annot=True, fmt='.1f', center=0, cmap='RdYlGn_r', cbar_kws={'label':'Δ AQI vs 2020'})
        plt.xlabel('Mes')
        plt.ylabel('Año')
        plt.title('Cambio mensual AQI ciudad vs 2020 (negativo = mejora)')
        plt.tight_layout()
        plt.savefig(os.path.join(figs_aqi_dir, 'AQI_city_monthly_change_vs2020.png'), dpi=150)
        plt.close()


Note: you may need to restart the kernel to use updated packages.


<Figure size 1000x400 with 0 Axes>

In [39]:
# Métricas mensuales (mean) por contaminante: comparativa 2020 vs 2024 vs 2025
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tables_dir = os.path.join('..','reports','tables')
figs_mm_dir = os.path.join('..','reports','figs','monthly_metrics')
os.makedirs(figs_mm_dir, exist_ok=True)

mm_path = os.path.join(tables_dir, 'metrics_monthly.csv')
mm = pd.read_csv(mm_path, parse_dates=['month'])
mm['year'] = mm['month'].dt.year
mm['month_num'] = mm['month'].dt.month

years = [2020, 2024, 2025]
mm_f = mm[mm['year'].isin(years)].copy()
metric = 'mean'

city_med = (mm_f.groupby(['pollutant','year','month_num'])[metric]
              .median()
              .reset_index()
              .rename(columns={metric: f'{metric}_median_city'}))
# Guardar tabla agregada
city_med.to_csv(os.path.join(tables_dir, f'metrics_monthly_city_median_{metric}.csv'), index=False)

pollutants = sorted(city_med['pollutant'].unique())
for pol in pollutants:
    sub = city_med[city_med['pollutant'] == pol]
    pivot = sub.pivot(index='month_num', columns='year', values=f'{metric}_median_city').sort_index()

    # Líneas por año
    plt.figure(figsize=(10,4))
    pivot.plot(marker='o')
    plt.title(f'{pol} mediana mensual (ciudad) - {metric} - 2020 vs 2024 vs 2025')
    plt.xlabel('Mes')
    plt.ylabel(f'{metric} (menor es mejor)')
    plt.xticks(range(1,13))
    plt.grid(True, alpha=0.3)
    plt.legend(title='Año')
    plt.tight_layout()
    plt.savefig(os.path.join(figs_mm_dir, f'metrics_monthly_{pol}_city_{metric}_comparison.png'), dpi=150)
    plt.close()

    # Mapa de calor: cambio vs 2020 (negativo = mejora)
    if 2020 in pivot.columns:
        delta = pivot.subtract(pivot[2020], axis=0)
        keep_cols = [c for c in [2024, 2025] if c in delta.columns]
        if keep_cols:
            deltaT = delta[keep_cols].T
            deltaT.index.name = 'year'
            # Guardar tabla tidy por contaminante
            delta_melt = deltaT.reset_index().melt(id_vars='year', var_name='month_num', value_name='delta_vs2020')
            delta_melt['pollutant'] = pol
            out_delta = os.path.join(tables_dir, f'metrics_monthly_{pol}_city_{metric}_change_vs2020.csv')
            delta_melt.to_csv(out_delta, index=False)

            plt.figure(figsize=(10,2.5))
            sns.heatmap(deltaT, annot=True, fmt='.2f', center=0, cmap='RdYlGn_r', cbar_kws={'label': f'Δ {metric} vs 2020'})
            plt.xlabel('Mes')
            plt.ylabel('Año')
            plt.title(f'{pol}: Cambio mensual vs 2020 ({metric}) (negativo = mejora)')
            plt.tight_layout()
            plt.savefig(os.path.join(figs_mm_dir, f'metrics_monthly_{pol}_city_{metric}_change_vs2020.png'), dpi=150)
            plt.close()


<Figure size 1000x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>